In [1]:
import subprocess

result = subprocess.check_call([
        'youtube-dl',
        '--no-playlist',
        '--extract-audio',
        '--audio-format', 'mp3',
        '--output', '/home/py/projects/tmp/tmp2.%(ext)s',
        '--cache-dir', '/tmp/youtube-dl',
        'IarsrX60bZw',])

In [2]:
result = subprocess.check_call([
        'sonic-annotator',
        '-d',
        'vamp:nnls-chroma:chordino:simplechord',
        '/home/py/projects/tmp/tmp2.mp3',
        '-w', 'csv',
        '--csv-force',
        '--force',
    ]) 
            

In [3]:
import csv

with open('/home/py/projects/tmp/tmp2_vamp_nnls-chroma_chordino_simplechord.csv', 'r') as f:
    reader = csv.reader(f)
    chords = list(reader)

print(chords)

[['0.000000000', 'N'], ['0.042666666', '1-m'], ['5.802666666', '8-m'], ['9.472000000', '9-'], ['13.098666666', '4-'], ['15.829333333', '11-'], ['17.066666666', '1-m'], ['20.437333333', '9-'], ['24.234666666', '4-'], ['25.429333333', '8-m'], ['28.245333333', '1-'], ['29.781333333', '11-'], ['35.157333333', '6-m'], ['37.034666666', '9-'], ['37.888000000', '11-'], ['38.442666666', '6-'], ['38.912000000', '11-'], ['41.728000000', '4-'], ['45.269333333', '11-'], ['47.061333333', '1-m'], ['48.000000000', '9-'], ['49.024000000', '11-'], ['52.565333333', '1-m'], ['55.168000000', '8-'], ['56.448000000', '1-'], ['58.453333333', '11-'], ['59.050666666', '1-'], ['60.757333333', '9-'], ['62.634666666', '11-'], ['66.773333333', '8-m'], ['67.754666666', '4-'], ['68.693333333', '1-m'], ['71.296000000', '6-'], ['71.722666666', '9-'], ['73.813333333', '1-m'], ['75.520000000', '11-'], ['78.805333333', '1-m'], ['79.274666666', '9-'], ['82.005333333', '11-'], ['85.504000000', '1-m'], ['88.917333333', '6-m'

In [8]:
from __future__ import division

import math

import numpy
import librosa
import scipy.spatial.distance
import scipy.signal

CHORDS = numpy.array([
    [1,0,0,0,1,0,0,1,0,0,0,0],
    [1,0,0,1,0,0,0,1,0,0,0,0],
    [0,1,0,0,0,1,0,0,1,0,0,0],
    [0,1,0,0,1,0,0,0,1,0,0,0],
    [0,0,1,0,0,0,1,0,0,1,0,0],
    [0,0,1,0,0,1,0,0,0,1,0,0],
    [0,0,0,1,0,0,0,1,0,0,1,0],
    [0,0,0,1,0,0,1,0,0,0,1,0],
    [0,0,0,0,1,0,0,0,1,0,0,1],
    [0,0,0,0,1,0,0,1,0,0,0,1],
    [1,0,0,0,0,1,0,0,0,1,0,0],
    [1,0,0,0,0,1,0,0,1,0,0,0],
    [0,1,0,0,0,0,1,0,0,0,1,0],
    [0,1,0,0,0,0,1,0,0,1,0,0],
    [0,0,1,0,0,0,0,1,0,0,0,1],
    [0,0,1,0,0,0,0,1,0,0,1,0],
    [1,0,0,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,1,0,0,1],
    [0,1,0,0,1,0,0,0,0,1,0,0],
    [1,0,0,0,1,0,0,0,0,1,0,0],
    [0,0,1,0,0,1,0,0,0,0,1,0],
    [0,1,0,0,0,1,0,0,0,0,1,0],
    [0,0,0,1,0,0,1,0,0,0,0,1],
    [0,0,1,0,0,0,1,0,0,0,0,1]
])

CHORD_NAMES = numpy.array([#"A", "Am",
               #"A#", "A#m",
               #"B", "Bm",
               "C", "Cm",
               "C#", "C#m",
               "D", "Dm",
               "D#", "D#m",
               "E", "Em",
               "F", "Fm",
               "F#", "F#m",
               "G", "Gm",
               "G#", "G#m",
               "A", "Am",
               "A#", "A#m",
        "B", "Bm"])

def analyse(filename, resample_to=2756, bt_hop_length=128,
            chroma_hop_length=512, chroma_n_fft=1024):
    samples, sampleRate = librosa.load(filename)
    length = float(len(samples))/sampleRate
    if resample_to:
        samples = librosa.resample(samples, sampleRate, resample_to)
        sampleRate = resample_to
    newSampleRate = 2756
    samples = librosa.resample(samples, sampleRate, newSampleRate)
    sampleRate = newSampleRate
    tempo, beats = librosa.beat.beat_track(samples, sampleRate,
                                           hop_length=bt_hop_length)
    beat_times = librosa.frames_to_time(beats, sampleRate,
                                        hop_length=bt_hop_length)
    chromagram = librosa.feature.chromagram(samples, sampleRate,
                                            hop_length=chroma_hop_length,
                                            n_fft=chroma_n_fft)
    chromagram = numpy.transpose(chromagram)
    distances = scipy.spatial.distance.cdist(chromagram, CHORDS, "cosine")
    chords = distances.argmin(axis=1)
    chords = scipy.signal.medfilt(chords, 11)
    chord_frames = numpy.array(numpy.where(numpy.diff(chords) != 0))
    chords = chords[chord_frames][0].astype(int)
    chord_times = librosa.frames_to_time(chord_frames, sampleRate,
                                         hop_length=chroma_hop_length,
                                         n_fft=chroma_n_fft)[0]
    chord_names = CHORD_NAMES[chords]
    return {"beats": list(beat_times),
            "chords": [{"chord": chord_name, "time": chord_time} for chord_name, chord_time in zip(chord_names, chord_times)],
"tempo": tempo}

In [9]:
print(analyse('/home/py/projects/tmp/tmp2.mp3'))

{'tempo': 0, 'chords': [{'time': 1.6719883889695211, 'chord': 'C'}, {'time': 1.8577648766328012, 'chord': 'Dm'}, {'time': 2.0435413642960811, 'chord': 'A'}, {'time': 5.9448476052249637, 'chord': 'Am'}, {'time': 6.1306240928882438, 'chord': 'G'}, {'time': 9.6603773584905657, 'chord': 'Em'}, {'time': 13.375907111756169, 'chord': 'F'}, {'time': 14.676342525399129, 'chord': 'Gm'}, {'time': 16.348330914368649, 'chord': 'G'}, {'time': 16.534107402031932, 'chord': 'E'}, {'time': 16.719883889695211, 'chord': 'Dm'}, {'time': 16.90566037735849, 'chord': 'E'}, {'time': 17.09143686502177, 'chord': 'G'}, {'time': 17.648766328011611, 'chord': 'E'}, {'time': 17.83454281567489, 'chord': 'G'}, {'time': 18.391872278664732, 'chord': 'A'}, {'time': 19.878084179970973, 'chord': 'Am'}, {'time': 20.063860667634252, 'chord': 'A'}, {'time': 20.435413642960814, 'chord': 'Am'}, {'time': 20.621190130624093, 'chord': 'A'}, {'time': 20.806966618287372, 'chord': 'Gm'}, {'time': 24.708272859216255, 'chord': 'F'}, {'t

/usr/local/lib/python3.5/dist-packages/scipy/spatial/distance.py:1772: RuntimeWarning: invalid value encountered in true_divide
  dm /= normsA.reshape(-1, 1)
